In [1]:
"""
**MORE ROBUST SOMA FINDNG ALGORITHM**
Purpose: Creates the table that will store the soma centroid from meshes used for testing

Pseudocode: 
1) Pull down the mesh
2) Create a folder where all of the mesh pieces will be stored
3) Run the soma extraction to get the soma meshes
4) Calculate the soma centers and save the distance from the predicted soma centers
5) Things then to save off:
- index of the soma (based on tehe number of somas found for that one mesh)
- the soma center
- Distance from predicted soma center
- Mesh of the soma
- Time it took
6) Delete all the temporary files that were used for that certain mesh
"""

'\n**MORE ROBUST SOMA FINDNG ALGORITHM**\nPurpose: Creates the table that will store the soma centroid from meshes used for testing\n\nPseudocode: \n1) Pull down the mesh\n2) Create a folder where all of the mesh pieces will be stored\n3) Run the soma extraction to get the soma meshes\n4) Calculate the soma centers and save the distance from the predicted soma centers\n5) Things then to save off:\n- index of the soma (based on tehe number of somas found for that one mesh)\n- the soma center\n- Distance from predicted soma center\n- Mesh of the soma\n- Time it took\n6) Delete all the temporary files that were used for that certain mesh\n'

In [2]:
# Testing 
import datajoint as dj
import numpy as np
m65 = dj.create_virtual_module('m65', 'microns_minnie65_01')
schema = dj.schema("microns_minnie65_01")
dj.config["display.limit"] = 30

import minfig
minnie = minfig.configure_minnie(return_virtual_module=True)

Connecting celiib@10.28.0.34:3306


In [3]:
segments = (m65.Mesh() & f'n_vertices  > {np.str(np.round(np.exp(12)).astype(np.int))}' & f'n_vertices  < {np.str(np.round(np.exp(15)).astype(np.int))}')
minnie.Decimation.proj(decimation_version='version') & segments.proj()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
73192496934525087,0,0.25
73404771398417561,0,0.25
73473215997272920,0,0.25
73543378515619332,0,0.25
73544203216881449,0,0.25
73548876073768090,0,0.25
73615190369126640,0,0.25
73685696619348325,0,0.25
73691056805396186,0,0.25
73754278724137980,0,0.25


# Testing Renaming Table

In [4]:
# new_table = dj.U("segment_id","version") & minnie.BaylorSegmentCentroid()

# @schema
# class BaylorTest(dj.Computed):
#     definition="""
#     ->new_table
#     ---
#     """

In [5]:
import cgal_Segmentation_Module as csm
from whole_neuron_classifier_datajoint_adapted import extract_branches_whole_neuron
import whole_neuron_classifier_datajoint_adapted as wcda 
import time
import trimesh
import numpy as np
import datajoint as dj
import os
from meshlab import Decimator , Poisson
from pathlib import Path
from pykdtree.kdtree import KDTree

In [6]:
m65.AllenMultiSomas() & "status > 0"

"segment_id segment id unique within each Segmentation, should be mirrored in Segment table.","status verification status (NULL=not checked, 0=ignore, 1=partial, 2=complete)"
77489456693007645,5
81639563398742139,2
82269790323894531,2
84106524699939179,3
86015414123380349,2
86988687602705632,2
89387891029279745,2
90153356676844854,2
90518669414951752,2
90939989466098088,2


# Getting the Segments That have multiple somas

In [7]:
minnie.Decimation() & (m65.AllenMultiSomas() & "status > 0").proj()

segment_id segment id unique within each Segmentation,version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),n_vertices,n_faces,mesh in-place path to the hdf5 (decimated) mesh file
77489456693007645,0,0.25,218373,438232,=BLOB=
81639563398742139,0,0.25,737715,1478740,=BLOB=
89387891029279745,0,0.25,577007,1155202,=BLOB=
90153356676844854,0,0.25,544744,1090856,=BLOB=
90518669414951752,0,0.25,147610,296288,=BLOB=
90939989466098088,0,0.25,369403,740740,=BLOB=
91208269699305759,0,0.25,1550148,3100434,=BLOB=
94031059578350884,0,0.25,1166108,2335958,=BLOB=
94531956582329737,0,0.25,305789,616274,=BLOB=
96147481821744505,0,0.25,110154,220298,=BLOB=


# Getting the Segments with One Soma

In [8]:
decimation_version = 0
decimation_ratio = 0.25
one_soma = (minnie.Decimation & (dj.U("segment_id") & (m65.AllenSegmentCentroid & "status=1").proj()) & "version=" + str(decimation_version))
one_soma

segment_id segment id unique within each Segmentation,version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),n_vertices,n_faces,mesh in-place path to the hdf5 (decimated) mesh file
77000105634561899,0,0.25,208308,417500,=BLOB=
77490624856971637,0,0.25,81815,163578,=BLOB=
77773542942770096,0,0.25,197032,394454,=BLOB=
77907065489896172,0,0.25,102538,205050,=BLOB=
78488706738250432,0,0.25,131596,272384,=BLOB=
78971460928405166,0,0.25,214377,429344,=BLOB=
80512770407472498,0,0.25,227270,454884,=BLOB=
80583139755390727,0,0.25,340287,681892,=BLOB=
81082729277563634,0,0.25,225506,451734,=BLOB=
81160726756366710,0,0.25,50916,101784,=BLOB=


# How to check if in Allen Soma segment exists

In [9]:
# Will Save for Later

# Helper Functions For Soma Extraction

In [10]:
"""
Checking the new validation checks
"""
def side_length_ratios(current_mesh):
    """
    Will compute the ratios of the bounding box sides
    To be later used to see if there is skewness
    """

    # bbox = current_mesh.bounding_box_oriented.vertices
    bbox = current_mesh.bounding_box_oriented.vertices
    x_axis_unique = np.unique(bbox[:,0])
    y_axis_unique = np.unique(bbox[:,1])
    z_axis_unique = np.unique(bbox[:,2])
    x_length = (np.max(x_axis_unique) - np.min(x_axis_unique)).astype("float")
    y_length = (np.max(y_axis_unique) - np.min(y_axis_unique)).astype("float")
    z_length = (np.max(z_axis_unique) - np.min(z_axis_unique)).astype("float")
    #print(x_length,y_length,z_length)
    #compute the ratios:
    xy_ratio = float(x_length/y_length)
    xz_ratio = float(x_length/z_length)
    yz_ratio = float(y_length/z_length)
    side_ratios = [xy_ratio,xz_ratio,yz_ratio]
    flipped_side_ratios = []
    for z in side_ratios:
        if z < 1:
            flipped_side_ratios.append(1/z)
        else:
            flipped_side_ratios.append(z)
    return flipped_side_ratios

def side_length_check(current_mesh,side_length_ratio_threshold=3):
    side_length_ratio_names = ["xy","xz","yz"]
    side_ratios = side_length_ratios(current_mesh)
    pass_threshold = [(k <= side_length_ratio_threshold) and
                      (k >= 1/side_length_ratio_threshold) for k in side_ratios]
    for i,(rt,truth) in enumerate(zip(side_ratios,pass_threshold)):
        if not truth:
            print(f"{side_length_ratio_names[i]} = {rt} ratio was beyong {side_length_ratio_threshold} multiplier")

    if False in pass_threshold:
        return False
    else:
        return True
# current_Pmesh = trimesh.load_mesh("./12346/12346_poisson_soma.off")
# print(side_length_ratios(current_mesh))
# side_length_check(current_mesh,side_length_ratio_threshold=1.55)



In [11]:
import random

def largest_mesh_piece(msh):
    mesh_splits_inner = msh.split(only_watertight=False)
    total_mesh_split_lengths_inner = [len(k.faces) for k in mesh_splits_inner]
    ordered_mesh_splits_inner = mesh_splits_inner[np.flip(np.argsort(total_mesh_split_lengths_inner))]
    return ordered_mesh_splits_inner[0]
def soma_volume_ratio(current_mesh):
    """
    bounding_box_oriented: rotates the box to be less volume
    bounding_box : does not rotate the box and makes it axis aligned
    
    ** checks to see if closed mesh and if not then make closed **
    """
    poisson_temp_folder = Path.cwd() / "Poisson_temp"
    poisson_temp_folder.mkdir(parents=True,exist_ok=True)
    with Poisson(poisson_temp_folder,overwrite=True) as Poisson_obj_temp:

        #get the largest piece
        lrg_mesh = largest_mesh_piece(current_mesh)
        if not lrg_mesh.is_watertight:
            print("Using Poisson Surface Reconstruction to make mesh watertight")
            #run the Poisson Surface reconstruction and get the largest piece
            new_mesh_inner,poisson_file_obj = Poisson_obj_temp(vertices=lrg_mesh.vertices,
                   faces=lrg_mesh.faces,
                   return_mesh=True,
                   delete_temp_files=True,
                   segment_id=random.randint(0,999999))
            lrg_mesh = largest_mesh_piece(new_mesh_inner)

        #turn the mesh into a closed mesh based on 

        ratio_val = lrg_mesh.bounding_box.volume/lrg_mesh.volume
    #     if ratio_val < 1:
    #         raise Exception("Less than 1 value in volume ratio computation")
    
    return ratio_val

def soma_volume_check(current_mesh,multiplier=8):
    ratio_val= soma_volume_ratio(current_mesh)
    print("Inside sphere validater: ratio_val = " + str(ratio_val))
    if np.abs(ratio_val) > multiplier:
        return False
    return True

# Functions that will extract the Somas

In [12]:
def extract_soma_center(segment_id,
                        current_mesh_verts,
                        current_mesh_faces,
                       outer_decimation_ratio= 0.25,
                        large_mesh_threshold = 60000,
                        large_mesh_threshold_inner = 40000,
                        soma_width_threshold = 0.32,
                        soma_size_threshold = 20000,
                       inner_decimation_ratio = 0.25,
                       volume_mulitplier=8,
                       side_length_ratio_threshold=3,
                       soma_size_threshold_max=192000, #this puts at 12000 once decimated
                       delete_files=True
                       ):    
    """
    Will extract the soma meshes (possible multiple) from
    a single mesh

    """
    
    global_start_time = time.time()

    #Adjusting the thresholds based on the decimations
    large_mesh_threshold = large_mesh_threshold*outer_decimation_ratio
    large_mesh_threshold_inner = large_mesh_threshold_inner*outer_decimation_ratio
    soma_size_threshold = soma_size_threshold*outer_decimation_ratio
    soma_size_threshold_max = soma_size_threshold_max*outer_decimation_ratio

    #adjusting for inner decimation
    soma_size_threshold = soma_size_threshold*inner_decimation_ratio
    soma_size_threshold_max = soma_size_threshold_max*inner_decimation_ratio
    print(f"Current Arguments Using (adjusted for decimation):\n large_mesh_threshold= {large_mesh_threshold}"
                 f" \nlarge_mesh_threshold_inner = {large_mesh_threshold_inner}"
                  f" \nsoma_size_threshold = {soma_size_threshold}"
                 f" \nsoma_size_threshold_max = {soma_size_threshold_max}"
                 f"\nouter_decimation_ratio = {outer_decimation_ratio}"
                 f"\ninner_decimation_ratio = {inner_decimation_ratio}")


    # ------------------------------


    temp_folder = f"./{segment_id}"
    temp_object = Path(temp_folder)
    #make the temp folder if it doesn't exist
    temp_object.mkdir(parents=True,exist_ok=True)

    #making the decimation and poisson objections
    Dec_outer = Decimator(outer_decimation_ratio,temp_folder,overwrite=True)
    Dec_inner = Decimator(inner_decimation_ratio,temp_folder,overwrite=True)
    Poisson_obj = Poisson(temp_folder,overwrite=True)

    #Step 1: Decimate the Mesh and then split into the seperate pieces
    new_mesh,output_obj = Dec_outer(vertices=current_mesh_verts,
             faces=current_mesh_faces,
             segment_id=segment_id,
             return_mesh=True,
             delete_temp_files=False)

    #preforming the splits of the decimated mesh

    mesh_splits = new_mesh.split(only_watertight=False)

    #get the largest mesh
    mesh_lengths = np.array([len(split.faces) for split in mesh_splits])


    total_mesh_split_lengths = [len(k.faces) for k in mesh_splits]
    ordered_mesh_splits = mesh_splits[np.flip(np.argsort(total_mesh_split_lengths))]
    list_of_largest_mesh = [k for k in ordered_mesh_splits if len(k.faces) > large_mesh_threshold]

    print(f"Total found significant pieces before Poisson = {list_of_largest_mesh}")

    #if no significant pieces were found then will use smaller threshold
    if len(list_of_largest_mesh)<=0:
        print(f"Using smaller large_mesh_threshold because no significant pieces found with {large_mesh_threshold}")
        list_of_largest_mesh = [k for k in ordered_mesh_splits if len(k.faces) > large_mesh_threshold/2]

    total_soma_list = []
    total_classifier_list = []
    total_poisson_list = []
    total_soma_list_sdf = []

    #start iterating through where go through all pieces before the poisson reconstruction
    no_somas_found_in_big_loop = 0
    for i,largest_mesh in enumerate(list_of_largest_mesh):
        print(f"----- working on large mesh #{i}: {largest_mesh}")

        somas_found_in_big_loop = False

        largest_file_name = str(output_obj.stem) + "_largest_piece.off"
        pre_largest_mesh_path = temp_object / Path(str(output_obj.stem) + "_largest_piece.off")
        pre_largest_mesh_path = pre_largest_mesh_path.absolute()
        print(f"pre_largest_mesh_path = {pre_largest_mesh_path}")
        # ******* This ERRORED AND CALLED OUR NERUON NONE: 77697401493989254 *********
        new_mesh_inner,poisson_file_obj = Poisson_obj(vertices=largest_mesh.vertices,
                   faces=largest_mesh.faces,
                   return_mesh=True,
                   mesh_filename=largest_file_name,
                   delete_temp_files=False)


        #splitting the Poisson into the largest pieces and ordering them
        mesh_splits_inner = new_mesh_inner.split(only_watertight=False)
        total_mesh_split_lengths_inner = [len(k.faces) for k in mesh_splits_inner]
        ordered_mesh_splits_inner = mesh_splits_inner[np.flip(np.argsort(total_mesh_split_lengths_inner))]

        list_of_largest_mesh_inner = [k for k in ordered_mesh_splits_inner if len(k.faces) > large_mesh_threshold_inner]
        print(f"Total found significant pieces AFTER Poisson = {list_of_largest_mesh_inner}")

        n_failed_inner_soma_loops = 0
        for j, largest_mesh_inner in enumerate(list_of_largest_mesh_inner):
            print(f"----- working on mesh after poisson #{j}: {largest_mesh_inner}")

            largest_mesh_path_inner = str(poisson_file_obj.stem) + "_largest_inner.off"

            #Decimate the inner poisson piece
            largest_mesh_path_inner_decimated,output_obj_inner = Dec_inner(
                                vertices=largest_mesh_inner.vertices,
                                 faces=largest_mesh_inner.faces,
                                mesh_filename=largest_mesh_path_inner,
                                 return_mesh=True,
                                 delete_temp_files=False)

            print(f"done exporting decimated mesh: {largest_mesh_path_inner}")

            faces = np.array(largest_mesh_path_inner_decimated.faces)
            verts = np.array(largest_mesh_path_inner_decimated.vertices)

            segment_id_new = int(str(segment_id) + f"{i}{j}")

            verts_labels, faces_labels, soma_value,classifier = wcda.extract_branches_whole_neuron(
                                    import_Off_Flag=False,
                                    segment_id=segment_id_new,
                                    vertices=verts,
                                     triangles=faces,
                                    pymeshfix_Flag=False,
                                     import_CGAL_Flag=False,
                                     return_Only_Labels=True,
                                     clusters=3,
                                     smoothness=0.2,
                                    soma_only=True,
                                    return_classifier = True
                                    )
            print(f"soma_sdf_value = {soma_value}")

            total_classifier_list.append(classifier)
            #total_poisson_list.append(largest_mesh_path_inner_decimated)

            # Save all of the portions that resemble a soma
            median_values = np.array([v["median"] for k,v in classifier.sdf_final_dict.items()])
            segmentation = np.array([k for k,v in classifier.sdf_final_dict.items()])

            #order the compartments by greatest to smallest
            sorted_medians = np.flip(np.argsort(median_values))
            print(f"segmentation[sorted_medians],median_values[sorted_medians] = {(segmentation[sorted_medians],median_values[sorted_medians])}")
            print(f"Sizes = {[classifier.sdf_final_dict[g]['n_faces'] for g in segmentation[sorted_medians]]}")

            valid_soma_segments_width = [g for g,h in zip(segmentation[sorted_medians],median_values[sorted_medians]) if ((h > soma_width_threshold)
                                                                and (classifier.sdf_final_dict[g]["n_faces"] > soma_size_threshold)
                                                                and (classifier.sdf_final_dict[g]["n_faces"] < soma_size_threshold_max))]
            valid_soma_segments_sdf = [h for g,h in zip(segmentation[sorted_medians],median_values[sorted_medians]) if ((h > soma_width_threshold)
                                                                and (classifier.sdf_final_dict[g]["n_faces"] > soma_size_threshold)
                                                                and (classifier.sdf_final_dict[g]["n_faces"] < soma_size_threshold_max))]

            print("valid_soma_segments_width")
            to_add_list = []
            to_add_list_sdf = []
            if len(valid_soma_segments_width) > 0:
                print(f"      ------ Found {len(valid_soma_segments_width)} viable somas: {valid_soma_segments_width}")
                somas_found_in_big_loop = True
                #get the meshes only if signfiicant length
                labels_list = classifier.labels_list

                for v,sdf in zip(valid_soma_segments_width,valid_soma_segments_sdf):
                    submesh_face_list = np.where(classifier.labels_list == v)[0]
                    soma_mesh = largest_mesh_path_inner_decimated.submesh([submesh_face_list],append=True)

                    
                    if side_length_check(soma_mesh,side_length_ratio_threshold) and soma_volume_check(soma_mesh,volume_mulitplier):
                        to_add_list.append(soma_mesh)
                        to_add_list_sdf.append(sdf)
                        
                    else:
                        print(f"--->This soma mesh was not added because it did not pass the sphere validation: {soma_mesh}")

                n_failed_inner_soma_loops = 0

            else:
                n_failed_inner_soma_loops += 1

            total_soma_list_sdf += to_add_list_sdf
            total_soma_list += to_add_list

            # --------------- KEEP TRACK IF FAILED TO FIND SOMA (IF TOO MANY FAILS THEN BREAK)
            if n_failed_inner_soma_loops >= 2:
                print("breaking inner loop because 2 soma fails in a row")
                break


        # --------------- KEEP TRACK IF FAILED TO FIND SOMA (IF TOO MANY FAILS THEN BREAK)
        if somas_found_in_big_loop == False:
            no_somas_found_in_big_loop += 1
            if no_somas_found_in_big_loop >= 2:
                print("breaking because 2 fails in a row in big loop")
                break

        else:
            no_somas_found_in_big_loop = 0





    """ IF THERE ARE MULTIPLE SOMAS THAT ARE WITHIN A CERTAIN DISTANCE OF EACH OTHER THEN JUST COMBINE THEM INTO ONE"""
    pairings = []
    for y,soma_1 in enumerate(total_soma_list):
        for z,soma_2 in enumerate(total_soma_list):
            if y<z:
                mesh_tree = KDTree(soma_1.vertices)
                distances,closest_node = mesh_tree.query(soma_2.vertices)

                if np.min(distances) < 4000:
                    pairings.append([y,z])


    #creating the combined meshes from the list
    total_soma_list_revised = []
    total_soma_list_revised_sdf = []
    if len(pairings) > 0:
        """
        Pseudocode: 
        Use a network function to find components

        """


        import networkx as nx
        new_graph = nx.Graph()
        new_graph.add_edges_from(pairings)
        grouped_somas = list(nx.connected_components(new_graph))

        somas_being_combined = []
        print(f"There were soma pairings: Connected components in = {grouped_somas} ")
        for comp in grouped_somas:
            comp = list(comp)
            somas_being_combined += list(comp)
            current_mesh = total_soma_list[comp[0]]
            for i in range(1,len(comp)):
                current_mesh += total_soma_list[comp[i]]

            total_soma_list_revised.append(current_mesh)
            #where can average all of the sdf values
            total_soma_list_revised_sdf.append(np.min(np.array(total_soma_list_sdf)[comp]))

        #add those that weren't combined to total_soma_list_revised
        leftover_somas = [total_soma_list[k] for k in range(0,len(total_soma_list)) if k not in somas_being_combined]
        leftover_somas_sdfs = [total_soma_list_sdf[k] for k in range(0,len(total_soma_list)) if k not in somas_being_combined]
        if len(leftover_somas) > 0:
            total_soma_list_revised += leftover_somas
            total_soma_list_revised_sdf += leftover_somas_sdfs
            
        print(f"Final total_soma_list_revised = {total_soma_list_revised}")
        print(f"Final total_soma_list_revised_sdf = {total_soma_list_revised_sdf}")


    if len(total_soma_list_revised) == 0:
        total_soma_list_revised = total_soma_list
        total_soma_list_revised_sdf = total_soma_list_sdf

    run_time = time.time() - global_start_time

    print(f"\n\n\n Total time for run = {time.time() - global_start_time}")

    #need to erase all of the temporary files ******
    #import shutil
    #shutil.rmtree(directory)

    """
    Need to delete all files in the temp folder *****
    """
    
    if delete_files:
        #now erase all of the files used
        from shutil import rmtree

        #remove the directory with the meshes
        rmtree(str(temp_object.absolute()))

        #removing the temporary files
        temp_folder = Path("./temp")
        temp_files = [x for x in temp_folder.glob('**/*')]
        seg_temp_files = [x for x in temp_files if str(segment_id) in str(x)]

        for f in seg_temp_files:
            f.unlink()

    #return total_soma_list, run_time
    return total_soma_list_revised,run_time,total_soma_list_revised_sdf



In [13]:
# # how to erase the temp files
# temp_folder = Path("./temp")
# temp_files = [x for x in temp_folder.glob('**/*')]
# seg_temp_files = [x for x in temp_files if str(ex_seg) in str(x)]
# seg_temp_files
# for f in seg_temp_files:
#     f.unlink()
# #str(temp_folder.absolute())

# Table Population

In [14]:
(minnie.Decimation() & (m65.AllenMultiSomas() & "status > 0")).proj()

segment_id segment id unique within each Segmentation,version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
77489456693007645,0,0.25
81639563398742139,0,0.25
89387891029279745,0,0.25
90153356676844854,0,0.25
90518669414951752,0,0.25
90939989466098088,0,0.25
91208269699305759,0,0.25
94031059578350884,0,0.25
94531956582329737,0,0.25
96147481821744505,0,0.25


In [15]:
decimation_version = 0
decimation_ratio = 0.25
import time

"""
- The error isn't computed anymore so disregarding these 
distance_from_prediction : double                   # the distance of the ALLEN predicted centroid soma center from the algorithms prediction
prediction_matching_index : int unsigned            # the soma index that was used to compute the error
"""

@schema
class BaylorSegmentCentroid(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    soma_index : tinyint unsigned #index given to this soma to account for multiple somas in one base semgnet
    ---
    centroid_x=NULL           : int unsigned                 # (EM voxels)
    centroid_y=NULL           : int unsigned                 # (EM voxels)
    centroid_z=NULL           : int unsigned                 # (EM voxels)
    n_vertices=NULL           : bigint                 #number of vertices
    n_faces=NULL            : bigint                  #number of faces
    soma_vertices=NULL        : longblob                # array of vertices
    soma_faces=NULL           : longblob                   # array of faces
    multiplicity=NULL         : tinyint unsigned             # the number of somas found for this base segment
    sdf=NULL                  : double                       # sdf width value for the soma
    max_side_ratio=NULL       : double                       # the maximum of the side length ratios used for check if soma
    bbox_volume_ratio=NULL    : double                       # ratio of bbox (axis aligned) volume to mesh volume to use for check if soma
    run_time=NULL : double                   # the amount of time to run (seconds)

    """
    
    #key_source = (minnie.Decimation & (dj.U("segment_id") & (m65.AllenSegmentCentroid & "status=1").proj()) & "version=" + str(decimation_version))
    #key_source = minnie.Decimation() & (m65.AllenMultiSomas() & "status > 0").proj()
    segments = (m65.Mesh() & f'n_vertices  > {np.str(np.round(np.exp(12)).astype(np.int))}' & f'n_vertices  < {np.str(np.round(np.exp(15)).astype(np.int))}')
    key_source =  minnie.Decimation.proj(decimation_version='version') & segments.proj() & "decimation_version=" + str(decimation_version)
    
    def make(self,key):
        #get the mesh data
        print(f"\n\n\n---- Working on {key['segment_id']} ----")
        
        new_mesh = (minnie.Decimation() & key).fetch1("mesh")
        current_mesh_verts,current_mesh_faces = new_mesh.vertices,new_mesh.faces
        
        segment_id = key["segment_id"]
        
        (total_soma_list, 
         run_time, 
         total_soma_list_sdf) = extract_soma_center(
                            segment_id,
                            current_mesh_verts,
                            current_mesh_faces,
                            outer_decimation_ratio= 0.25,
                            large_mesh_threshold = 60000,
                            large_mesh_threshold_inner = 40000,
                            soma_width_threshold = 0.32,
                            soma_size_threshold = 20000,
                           inner_decimation_ratio = 0.25,
                           volume_mulitplier=7,
                           side_length_ratio_threshold=3,
                            soma_size_threshold_max=192000,
                            delete_files=True
        )
        
        print(f"Run time was {run_time} \n    total_soma_list = {total_soma_list}"
             f"\n    with sdf values = {total_soma_list_sdf}")
        
        #check if soma list is empty and did not find soma
        if len(total_soma_list) <= 0:
            print("There were no somas found for this mesh so just writing empty data")
            insert_dict = dict(key,
                              soma_index=0,
                              centroid_x=None,
                               centroid_y=None,
                               centroid_z=None,
                               #distance_from_prediction=None,
                               #prediction_matching_index = None,
                               n_vertices=0,
                               n_faces=0,
                               soma_vertices=None,
                               soma_faces=None,
                               multiplicity=0,
                               sdf = None,
                               max_side_ratio = None,
                               bbox_volume_ratio = None,
                               run_time=run_time
                              )
            
            #raise Exception("to prevent writing because none were found")
            self.insert1(insert_dict,skip_duplicates=True)
            return
        
        #if there is one or more soma found, get the volume and side length checks
        max_side_ratio =  [np.max(side_length_ratios(m)) for m in total_soma_list]
        bbox_volume_ratio =  [soma_volume_ratio(m) for m in total_soma_list]
        dicts_to_insert = []
        
        
        """        
        #DON'T NEED THE ERROR PREDICTION ANYMORE
        new_array = (m65.AllenMultiSomas.Centroids()  & key).fetch("centroid_x","centroid_y","centroid_z")
        soma_ids = (m65.AllenMultiSomas.Centroids()  & key).fetch("soma_id")
        allen_centroid_prediction = np.array(new_array).T
        allen_centroid_prediction
        #print("soma_ids = " + str(soma_ids))
        from pykdtree.kdtree import KDTree
        mesh_tree = KDTree(allen_centroid_prediction)
        """
        
        
        

        for i,(current_soma,soma_sdf,sz_ratio,vol_ratio) in enumerate(zip(total_soma_list,total_soma_list_sdf,max_side_ratio,bbox_volume_ratio)):
            print("Trying to write off file")
            """ Currently don't need to export the meshes
            current_soma.export(f"{key['segment_id']}/{key['segment_id']}_soma_{i}.off")
            """
            auto_prediction_center = np.mean(current_soma.vertices,axis=0) / np.array([4,4,40])
            auto_prediction_center = auto_prediction_center.astype("int")
            print(f"Predicted Coordinates are {auto_prediction_center}")

        #             distances,closest_node = mesh_tree.query(auto_prediction_center.reshape(1,3))
        #             error_distance = distances[0]
        #             prediction_matching_index = soma_ids[closest_node[0]] #closest nodes and the distances


            insert_dict = dict(key,
                              soma_index=i+1,
                              centroid_x=auto_prediction_center[0],
                               centroid_y=auto_prediction_center[1],
                               centroid_z=auto_prediction_center[2],
        #                                distance_from_prediction=error_distance,
        #                                prediction_matching_index = prediction_matching_index,
                               n_vertices = len(current_soma.vertices),
                               n_faces = len(current_soma.faces),
                               soma_vertices=current_soma.vertices,
                               soma_faces=current_soma.faces,
                               multiplicity=len(total_soma_list),
                               sdf = np.round(soma_sdf,3),
                               max_side_ratio = np.round(sz_ratio,3),
                               bbox_volume_ratio = np.round(vol_ratio,3),
                               run_time=np.round(run_time,4)
                              )



            dicts_to_insert.append(insert_dict)
        
        #raise Exception("to prevent writing")

#         if len(total_soma_list) != len(soma_ids):
#             raise Exception("to prevent writing SOMAS NOT EQUAL TO That registered in datase")

        self.insert(dicts_to_insert,skip_duplicates=True)

In [16]:
#(SomaCentroidValidation & "segment_id=90231147459666747").delete()
#(schema.jobs & "table_name='__baylor_segment_centroid'").delete()
#m65.SomaCentroidValidation()

In [ ]:
"""
Actually will do the population
"""

#(schema.jobs & "table_name='__whole_auto_annotations_label_clusters3'")#.delete()
dj.config["enable_python_native_blobs"] = True

import time
start_time = time.time()
BaylorSegmentCentroid.populate(reserve_jobs=True)
print(f"Total time for BaylorSegmentCentroid populate = {time.time() - start_time}")




---- Working on 79183941617083896 ----
Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 15000.0 
large_mesh_threshold_inner = 10000.0 
soma_size_threshold = 1250.0 
soma_size_threshold_max = 12000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Soma/79183941617083896/neuron_79183941617083896.off -o /notebooks/Platinum_Soma/79183941617083896/neuron_79183941617083896_decimated.off -s /notebooks/Platinum_Soma/79183941617083896/decimation_meshlab_25404660.mls
Total found significant pieces before Poisson = [<trimesh.Trimesh(vertices.shape=(10824, 3), faces.shape=(23142, 3))>]
----- working on large mesh #0: <trimesh.Trimesh(vertices.shape=(10824, 3), faces.shape=(23142, 3))>
pre_largest_mesh_path = /notebooks/Platinum_Soma/79183941617083896/neuron_79183941617083896_decimated_largest_piece.off
IN INPUT FILE VALIDA

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!


Total found significant pieces AFTER Poisson = [<trimesh.Trimesh(vertices.shape=(12549, 3), faces.shape=(25162, 3))>]
----- working on mesh after poisson #0: <trimesh.Trimesh(vertices.shape=(12549, 3), faces.shape=(25162, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Soma/79183941617083896/neuron_79183941617083896_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Soma/79183941617083896/neuron_79183941617083896_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Platinum_Soma/79183941617083896/decimation_meshlab_25720678.mls
done exporting decimated mesh: neuron_79183941617083896_decimated_largest_piece_poisson_largest_inner.off
1) Starting: Mesh importing and Pymesh fix
loading mesh from vertices and triangles array
1) Finished: Mesh importing and Pymesh fix: 0.0002300739288330078
2) Staring: Generating CGAL segmentation for neuron
Done writing O

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!


Total found significant pieces AFTER Poisson = [<trimesh.Trimesh(vertices.shape=(47268, 3), faces.shape=(94540, 3))>]
----- working on mesh after poisson #0: <trimesh.Trimesh(vertices.shape=(47268, 3), faces.shape=(94540, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Soma/79184423056585858/neuron_79184423056585858_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Soma/79184423056585858/neuron_79184423056585858_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Platinum_Soma/79184423056585858/decimation_meshlab_2585765.mls
done exporting decimated mesh: neuron_79184423056585858_decimated_largest_piece_poisson_largest_inner.off
1) Starting: Mesh importing and Pymesh fix
loading mesh from vertices and triangles array
1) Finished: Mesh importing and Pymesh fix: 0.00023436546325683594
2) Staring: Generating CGAL segmentation for neuron
Done writing O

...

Total found significant pieces AFTER Poisson = [<trimesh.Trimesh(vertices.shape=(17943, 3), faces.shape=(35946, 3))>]
----- working on mesh after poisson #0: <trimesh.Trimesh(vertices.shape=(17943, 3), faces.shape=(35946, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Soma/79315127903739295/neuron_79315127903739295_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Soma/79315127903739295/neuron_79315127903739295_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Platinum_Soma/79315127903739295/decimation_meshlab_25545211.mls
done exporting decimated mesh: neuron_79315127903739295_decimated_largest_piece_poisson_largest_inner.off
1) Starting: Mesh importing and Pymesh fix
loading mesh from vertices and triangles array
1) Finished: Mesh importing and Pymesh fix: 0.0006358623504638672
2) Staring: Generating CGAL segmentation for neuron
Done writing O

face_normals all zero, ignoring!
face_normals all zero, ignoring!


removed temporary input file: /notebooks/Platinum_Soma/Poisson_temp/neuron_179594.off
removed temporary output file: /notebooks/Platinum_Soma/Poisson_temp/neuron_179594_poisson.off
/notebooks/Platinum_Soma/Poisson_temp/poisson_658381.mls is being deleted....
Inside sphere validater: ratio_val = 26.221596051422296
--->This soma mesh was not added because it did not pass the sphere validation: <trimesh.Trimesh(vertices.shape=(1455, 3), faces.shape=(2849, 3))>
Using Poisson Surface Reconstruction to make mesh watertight
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Soma/Poisson_temp/neuron_290948.off -o /notebooks/Platinum_Soma/Poisson_temp/neuron_290948_poisson.off -s /notebooks/Platinum_Soma/Poisson_temp/poisson_355585.mls


face_normals all zero, ignoring!
face_normals all zero, ignoring!


removed temporary input file: /notebooks/Platinum_Soma/Poisson_temp/neuron_290948.off
removed temporary output file: /notebooks/Platinum_Soma/Poisson_temp/neuron_290948_poisson.off
/notebooks/Platinum_Soma/Poisson_temp/poisson_355585.mls is being deleted....
Inside sphere validater: ratio_val = 19.489574114476792
--->This soma mesh was not added because it did not pass the sphere validation: <trimesh.Trimesh(vertices.shape=(822, 3), faces.shape=(1638, 3))>
Using Poisson Surface Reconstruction to make mesh watertight
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Soma/Poisson_temp/neuron_33272.off -o /notebooks/Platinum_Soma/Poisson_temp/neuron_33272_poisson.off -s /notebooks/Platinum_Soma/Poisson_temp/poisson_791338.mls
removed temporary input file: /notebooks/Platinum_Soma/Poisson_temp/neuron_33272.off
removed temporary output file: /notebooks/Platinum_Soma/Poisson_temp/neuron_33272_poisson.off


In [ ]:
BaylorSegmentCentroid()new_coord = np.array([787264,901589,779901]) / np.array([4,4,40])
# new_coord = new_coord.astype("int")
# print(f"x {new_coord[0]}, y {new_coord[1]}, z {new_coord[2]}")
# # good coordinates for 90231147459666747: x 196816, y 225397, z 19497

In [ ]:
# new_coord = np.array([787264,901589,779901]) / np.array([4,4,40])
# new_coord = new_coord.astype("int")
# print(f"x {new_coord[0]}, y {new_coord[1]}, z {new_coord[2]}")
# # good coordinates for 90231147459666747: x 196816, y 225397, z 19497

In [ ]:
# dj.config["display.limit"] = 100
# BaylorSegmentCentroid().drop()